In [26]:
import os
import pandas as pd
import glob
import openpyxl
# from openpyxl import load_workbook
from openpyxl.styles import Font, NamedStyle, Alignment
from openpyxl.utils.dataframe import dataframe_to_rows

In [27]:
def get_excel_files(user_name: str) -> list:
    all_files = os.listdir(os.path.join("C:\\Users","Marcos","Desktop", "Meses"))
    excel_files = []
    for file in all_files:
        if file.endswith(".xlsx"):
            excel_files.append("C:/Users/Marcos/Desktop/Meses/" + file)
    
    return excel_files

In [28]:
cuits = pd.read_excel('C:/Users/Marcos/Desktop/CUITS.xlsx')
# file_paths = glob.glob('C:/Users/Marcos/Desktop/Meses/*.xlsx')
file_paths = get_excel_files(1)

In [30]:
#from openpyxl.utils.dataframe import dataframe_to_rows
style = NamedStyle(name="Style")
style.font = Font(name="Calibri", size=11) 
i=0
for c in range(len(cuits)):
    if i != len(file_paths):
        alldata_emitidos = pd.DataFrame()
        alldata_recibidos = pd.DataFrame()
        for file_path in file_paths:
            if str(cuits.iloc[c,1]) in file_path:
                if 'Emitidos' in file_path:
                    alldata_emitidos = pd.concat([alldata_emitidos, pd.read_excel(file_path,skiprows=1)], axis=0, ignore_index=True).reset_index(drop=True)
                    # alldata_emitidos = alldata_emitidos.sort_values(by='row_number', ascending=False).reset_index(drop=True)
                else:
                    alldata_recibidos = pd.concat([alldata_recibidos, pd.read_excel(file_path,skiprows=1)], axis=0, ignore_index=True).reset_index(drop=True)  
                    # alldata_recibidos = alldata_recibidos.sort_values(by='row_number', ascending=False).reset_index(drop=True)
                i+=1
        if len(alldata_emitidos) > 0:
            data_nueva = alldata_emitidos[['Fecha', 'Tipo', 'Número Desde', 'Denominación Receptor', 'Imp. Total']].sort_values(by='Fecha', ascending=True)
            excel_filepath = f'C:/Users/Marcos/Desktop/Oficina/Monotributo/{cuits.iloc[c, 0]}.xlsx'
            data_excel = pd.read_excel(excel_filepath, sheet_name="COMPRAS NUEVO", usecols="A:E", skiprows=5)
            data_excel = data_excel.append(data_nueva, ignore_index=True)
            wb = openpyxl.load_workbook(excel_filepath)
            sheet = wb['VENTAS NUEVO']
            for row in sheet.iter_rows(min_row=7, max_row=sheet.max_row, min_col=1, max_col=5):
                for cell in row:
                    cell.value = None
            for row_idx, row in enumerate(openpyxl.utils.dataframe.dataframe_to_rows(data_excel, index=False, header=False), 7):
                for col_idx, value in enumerate(row, 1):
                    sheet.cell(row=row_idx, column=col_idx, value=value)
            wb.save(excel_filepath)
        if len(alldata_recibidos) > 0:
            data_nueva = alldata_recibidos[['Fecha', 'Tipo', 'Número Desde', 'Denominación Emisor', 'Imp. Total']].sort_values(by='Fecha', ascending=True)
            excel_filepath = f'C:/Users/Marcos/Desktop/Oficina/Monotributo/{cuits.iloc[c, 0]}.xlsx'
            data_excel = pd.read_excel(excel_filepath, sheet_name="COMPRAS NUEVO", usecols="A:E", skiprows=5)
            data_excel = data_excel.append(data_nueva, ignore_index=True)
            wb = openpyxl.load_workbook(excel_filepath)
            sheet = wb['COMPRAS NUEVO']
            for row in sheet.iter_rows(min_row=7, max_row=sheet.max_row, min_col=1, max_col=5):
                for cell in row:
                    cell.value = None
            for row_idx, row in enumerate(openpyxl.utils.dataframe.dataframe_to_rows(data_excel, index=False, header=False), 7):
                for col_idx, value in enumerate(row, 1):
                    sheet.cell(row=row_idx, column=col_idx, value=value)
            wb.save(excel_filepath)
    else:
        break


C:\Users\Marcos\AppData\Local\Temp\ipykernel_3116\2187969739.py:22: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data_excel = data_excel.append(data_nueva, ignore_index=True)
C:\Users\Marcos\AppData\Local\Temp\ipykernel_3116\2187969739.py:36: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data_excel = data_excel.append(data_nueva, ignore_index=True)


In [23]:
if(length(all_data_emitidos) > 0){
    data_final <- arrange(subset(all_data_emitidos, select = c('Fecha', 'Tipo', 'Número Desde', 'Denominación Receptor', 'Imp. Total')), desc(row_number()))
    #colnames(data_final) <- c("Fecha", "Tipo", "Nro Factura", "Denom. Receptor", "Imp. Total")
    data_final$Fecha <- as.Date(data_final$Fecha, format = "%d/%m/%Y")
    data_final <- data_final[order(data_final$Fecha), ]
    data_final$Fecha <- format(data_final$Fecha, format = "%d/%m/%Y")
    
    file_path_a <- paste0("C:/Users/Marcos/Desktop/Oficina/Monotributo/",clientes_cuit[c,1],".xlsx")
    wb <- loadWorkbook(file_path_a)
    excel_cliente <- read_xlsx(file_path_a, sheet = "VENTAS NUEVO", range = cell_cols(1:5))
    size <- nrow(excel_cliente)
    excel_cliente$Fecha <- format(excel_cliente$Fecha, format = "%d/%m/%Y")
    excel_cliente <- rbind(excel_cliente,data_final)
    writeData(wb, sheet = "VENTAS NUEVO", x = excel_cliente, startRow = 6, startCol = 1)
    
    for(i in 1:nrow(data_final)){
      b=size+6+i
      addStyle(wb, sheet = "VENTAS NUEVO", rows = b, cols = 1:5, style = createStyle(fontName = "Calibri", fontSize = 11))
    }
    addStyle(wb, sheet = "VENTAS NUEVO", rows = 7:(nrow(excel_cliente)+7), cols = 3, style =  createStyle(halign = "center", fontName = "Calibri", fontSize = 11))
    saveWorkbook(wb, file_path_a, overwrite = TRUE)
  }

range(0, 88)

In [24]:
data_excel = pd.read_excel(excel_filepath, sheet_name="COMPRAS NUEVO", usecols="A:E", skiprows=5)

In [25]:
data_excel.head(10)

,Fecha,Tipo,Nro Desde,Denom. Emisor,Imp. Total
0,2022-01-04,6 - Factura B,5892,MANSILLA E HIJO S.R.L.,64798.22
1,2022-01-07,6 - Factura B,34304,DISTRIBUIDORA DENVER S.R.L.,1941.30
2,2022-01-07,6 - Factura B,6801,TTC AUTO ARGENTINA S A,12993.00
3,2022-01-09,6 - Factura B,16360,GRAELLS NELSON ANTONIO,15500.00
4,2022-01-10,6 - Factura B,18850,GRAELLS NELSON ANTONIO,9500.00
5,2022-01-11,6 - Factura B,2339,VESPASIANI AUTOMOTORES SA,3700000.00
6,2022-01-21,15 - Recibo C,372268,ASOCIACION MUTUAL DE PROFESIONALES PARA LA ASI...,1700.00
7,2022-01-25,6 - Factura B,316007,ALVAREZ CABLE HOGAR SOCIEDAD ANONIMA,2725.00
8,2022-01-31,11 - Factura C,380,PIDELLO GABRIEL ALEJANDRO,2000.00
9,2022-01-31,1 - Factura A,5727222,TELECOM ARGENTINA SOCIEDAD ANONIMA,6827.62
